In [1]:
import pandas as pd

alert_path = "./SecurityAlert.csv"
incident_path = "./SecurityIncident.csv"

security_alert = pd.read_csv(alert_path)
security_incident = pd.read_csv(incident_path)


# sort by time
security_incident["TimeGenerated [UTC]"] = pd.to_datetime(security_incident["TimeGenerated [UTC]"])
security_incident = security_incident.sort_values(by="TimeGenerated [UTC]")

security_alert["TimeGenerated [UTC]"] = pd.to_datetime(security_alert["TimeGenerated [UTC]"])
security_alert = security_alert.sort_values(by="TimeGenerated [UTC]")

In [2]:
security_alert.columns

Index(['TenantId', 'TimeGenerated [UTC]', 'DisplayName', 'AlertName',
       'AlertSeverity', 'Description', 'ProviderName', 'VendorName',
       'VendorOriginalId', 'SystemAlertId', 'ResourceId', 'SourceComputerId',
       'AlertType', 'ConfidenceLevel', 'ConfidenceScore', 'IsIncident',
       'StartTime [UTC]', 'EndTime [UTC]', 'ProcessingEndTime [UTC]',
       'RemediationSteps', 'ExtendedProperties', 'Entities', 'SourceSystem',
       'WorkspaceSubscriptionId', 'WorkspaceResourceGroup', 'ExtendedLinks',
       'ProductName', 'ProductComponentName', 'AlertLink', 'Status',
       'CompromisedEntity', 'Tactics', 'Techniques', 'SubTechniques', 'Type'],
      dtype='object')

In [14]:
import json


# 1. Filter by LATEST label
filtered_incidents = []
for i, incident in security_incident.iterrows():
    labels = json.loads(incident['Labels'])
    # check if the incident is the latest in one line
    is_latest = any([label['labelName'] == 'LATEST' for label in labels])
    if is_latest:
        filtered_incidents.append(incident)

filtered_incidents = pd.DataFrame(filtered_incidents)

# unique incident numbers
incident_numbers = filtered_incidents['IncidentNumber'].unique()


In [15]:
# 2. for each incdent number, get the latest incident generated and put in a new dataframe
latest_incidents = []
for incident_number in incident_numbers:
    incident = filtered_incidents[filtered_incidents['IncidentNumber'] == incident_number]
    latest_incident = incident.iloc[-1]
    latest_incidents.append(latest_incident)

latest_incidents = pd.DataFrame(latest_incidents)
len(latest_incidents)

13

In [16]:
# get earliest start time and latest end time

earliest_start_time = latest_incidents['FirstActivityTime [UTC]'].min()
earliest_start_time

# sort and print start time
latest_incidents = latest_incidents.sort_values(by='FirstActivityTime [UTC]')
# latest_incidents[['IncidentNumber', 'Title', 'FirstActivityTime [UTC]']]
# print
for i, incident in latest_incidents.iterrows():
    print(incident['FirstActivityTime [UTC]'], incident['Title'])

6/20/2024, 8:51:07.052 AM Human-operated ransomware attack was launched from a compromised asset (attack disruption)
6/26/2024, 11:57:25.302 AM Multi-stage incident involving Execution & Discovery on one endpoint
6/26/2024, 3:49:16.784 PM Multi-stage incident involving Defense evasion & Discovery on one endpoint
6/27/2024, 2:25:58.353 PM Multi-stage incident involving Privilege escalation on multiple endpoints reported by multiple sources
6/4/2024, 12:21:22.478 PM BEC financial fraud attack was launched from a compromised account (attack disruption)
7/1/2024, 3:01:28.000 PM Multi-stage incident involving Initial access & Lateral movement on multiple endpoints reported by multiple sources
7/13/2024, 9:17:30.965 AM Multi-stage incident involving Initial access & Discovery on one endpoint
7/17/2024, 10:49:35.108 AM BEC financial fraud attack was launched from a compromised account (attack disruption)
7/22/2024, 8:18:18.418 AM SAP financial process manipulation (attack disruption)
7/23/202

In [18]:
latest_incidents.iloc[-2]

TenantId                                e34d562e-ef12-4c4e-9bc0-7c6ae357c015
TimeGenerated [UTC]                               2024-08-17 22:40:19.079000
IncidentName                            909f13db-13ce-4584-a4b5-6aa0e67397c6
Title                      Multi-stage incident involving Initial access ...
Description                                                              NaN
Severity                                                                High
Status                                                                   New
Classification                                                           NaN
ClassificationComment                                                    NaN
ClassificationReason                                                     NaN
Owner                      {"objectId":null,"email":null,"assignedTo":"Mi...
ProviderName                                                   Microsoft XDR
ProviderIncidentId                                                       801

In [ ]:
Multi-stage incident involving Initial access &amp; Collection on multiple endpoints reported by multiple sources

In [8]:
latest_incidents = latest_incidents.drop_duplicates(subset=['Title'])
for i, incident in latest_incidents.iterrows():
    print( incident['Title'],)

len(latest_incidents)

Human-operated ransomware attack was launched from a compromised asset (attack disruption)
Multi-stage incident involving Execution & Discovery on one endpoint
Multi-stage incident involving Defense evasion & Discovery on one endpoint
Multi-stage incident involving Privilege escalation on multiple endpoints reported by multiple sources
BEC financial fraud attack was launched from a compromised account (attack disruption)
Multi-stage incident involving Initial access & Lateral movement on multiple endpoints reported by multiple sources
Multi-stage incident involving Initial access & Discovery on one endpoint
SAP financial process manipulation (attack disruption)
Multi-stage incident involving Initial access & Collection on multiple endpoints reported by multiple sources


In [12]:
titles = [
    "BEC financial fraud attack was launched from a compromised account (attack disruption)",
    "BEC financial fraud attack was launched from a compromised account (attack disruption)",
    "Human-operated ransomware attack was launched from a compromised asset (attack disruption)",
    "Multi-stage incident involving Initial access & Collection on multiple endpoints reported by multiple sources",
    "Multi-stage incident involving Initial access & Discovery on one endpoint",
    "Multi-stage incident involving Initial access & Lateral movement on multiple endpoints reported by multiple sources",
    "Multi-stage incident involving Privilege escalation on multiple endpoints reported by multiple sources",
    "SAP financial process manipulation (attack disruption)"
]

# if not in the list, print special message
for i, incident in latest_incidents.iterrows():
    if incident['Title'] not in titles:
        print(incident['Title'], " - Special message", incident['FirstActivityTime [UTC]'])
    else:
        print(incident['Title'], incident['FirstActivityTime [UTC]'])
        del titles[titles.index(incident['Title'])]


titles

Human-operated ransomware attack was launched from a compromised asset (attack disruption) 6/20/2024, 8:51:07.052 AM
Multi-stage incident involving Execution & Discovery on one endpoint  - Special message 6/26/2024, 11:57:25.302 AM
Multi-stage incident involving Defense evasion & Discovery on one endpoint  - Special message 6/26/2024, 3:49:16.784 PM
Multi-stage incident involving Privilege escalation on multiple endpoints reported by multiple sources 6/27/2024, 2:25:58.353 PM
BEC financial fraud attack was launched from a compromised account (attack disruption) 6/4/2024, 12:21:22.478 PM
Multi-stage incident involving Initial access & Lateral movement on multiple endpoints reported by multiple sources 7/1/2024, 3:01:28.000 PM
Multi-stage incident involving Initial access & Discovery on one endpoint 7/13/2024, 9:17:30.965 AM
SAP financial process manipulation (attack disruption) 7/22/2024, 8:18:18.418 AM
Multi-stage incident involving Initial access & Collection on multiple endpoints rep

['BEC financial fraud attack was launched from a compromised account (attack disruption)']

In [75]:
missing_titles

,TenantId,TimeGenerated [UTC],IncidentName,Title,Description,Severity,Status,Classification,ClassificationComment,ClassificationReason,...,AlertIds,BookmarkIds,Comments,Tasks,Labels,IncidentUrl,AdditionalData,ModifiedBy,SourceSystem,Type
769,e34d562e-ef12-4c4e-9bc0-7c6ae357c015,2024-07-12 22:41:40.978,037c9652-18db-49ab-9b7d-9d8313408909,Multi-stage incident involving Execution & Dis...,NaN,Informational,Active,NaN,NaN,NaN,...,"[""b1a0fd16-05e4-7bd1-5b5b-1baf8c63d8e0"",""b67ce...",[],"[{""message"":""config"",""createdTimeUtc"":""2024-06...",[],"[{""labelName"":""Backdoor"",""labelType"":""User""},{...",https://portal.azure.com/#asset/Microsoft_Azur...,"{""alertsCount"":11,""bookmarksCount"":0,""comments...",External application - Mimik Emails - AlpineSk...,Azure,SecurityIncident
583,e34d562e-ef12-4c4e-9bc0-7c6ae357c015,2024-08-01 12:07:27.002,11ea298e-e20e-404b-b5ca-f7276c94de7c,Multi-stage incident involving Defense evasion...,NaN,Medium,Closed,Undetermined,NaN,NaN,...,"[""197c2262-2cd9-4ca0-4c33-add856f8af63"",""d368a...",[],"[{""message"":""Fileless demo"",""createdTimeUtc"":""...",[],"[{""labelName"":""Fileless"",""labelType"":""User""},{...",https://portal.azure.com/#asset/Microsoft_Azur...,"{""alertsCount"":4,""bookmarksCount"":0,""commentsC...",u101@a.alpineskihouse.co,Azure,SecurityIncident


In [69]:
len(latest_incidents)

13

In [60]:
def get_alert_titles(alert_ids: list):
    alert_titles = []
    for alert_id in alert_ids:
        alert = security_alert[security_alert['SystemAlertId'] == alert_id]
        if len(alert) > 0:
            alert_titles.append(alert.iloc[0]['AlertName'])
    return alert_titles


existing_alerts = []
for i, incident in latest_incidents.iterrows():
    print(f"Num {incident['IncidentNumber']}, Title:{incident['Title']} {incident['FirstActivityTime [UTC]']}, {len(json.loads(incident['AlertIds']))}")
    atitles = set(get_alert_titles(json.loads(incident['AlertIds'])))
    print(len(atitles))
    if len(atitles) == 0:
        print(f"Num {incident['IncidentNumber']} has no alerts")
        continue
    for inum, patitiles in existing_alerts:
        inter = atitles.intersection(patitiles)
        if len(inter) == min(len(atitles), len(patitiles)):
            print(patitiles)
            print(atitles)
            print(f"Num {incident['IncidentNumber']} and {inum} have common alerts")
    
    existing_alerts.append((incident['IncidentNumber'], atitles))
        

Num 38, Title:Multi-stage incident involving Defense evasion & Discovery on one endpoint 6/26/2024, 3:49:16.784 PM, 4
3
Num 5, Title:Human-operated ransomware attack was launched from a compromised asset (attack disruption) 6/20/2024, 8:51:07.052 AM, 71
0
Num 5 has no alerts
Num 3, Title:BEC financial fraud attack was launched from a compromised account (attack disruption) 6/4/2024, 12:21:22.478 PM, 5
0
Num 3 has no alerts
Num 39, Title:Multi-stage incident involving Privilege escalation on multiple endpoints reported by multiple sources 6/27/2024, 2:25:58.353 PM, 48
32
Num 34, Title:Multi-stage incident involving Execution & Discovery on one endpoint 6/26/2024, 11:57:25.302 AM, 11
10
Num 55, Title:Multi-stage incident involving Initial access & Lateral movement on multiple endpoints reported by multiple sources 7/1/2024, 3:01:28.000 PM, 28
22
Num 122, Title:Multi-stage incident involving Initial access & Discovery on one endpoint 7/13/2024, 9:17:30.965 AM, 9
8
Num 134, Title:BEC finan

In [37]:
# unique incident numbers
incident_numbers = filtered_incidents['IncidentNumber'].unique()

# get all incidents with the first incident number
first_incident = filtered_incidents[filtered_incidents['IncidentNumber'] == incident_numbers[0]]
for _, f  in first_incident.iterrows():
    print(f['FirstActivityTime [UTC]'])

6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM
6/26/2024, 3:49:16.784 PM


In [32]:

# get first entry from each unique incident number
unique_incidents = filtered_incidents.groupby('IncidentNumber').first()

# back to dataframe
unique_incidents = unique_incidents.reset_index()

for i, incident in unique_incidents.iterrows():
    print(f"Incident number {incident['IncidentNumber']}: {incident['FirstActivityTime [UTC]']}, {len(json.loads(incident['AlertIds']))}")

Incident number 3: 6/4/2024, 12:21:22.478 PM, 5
Incident number 5: 6/20/2024, 8:51:07.052 AM, 71
Incident number 34: 6/26/2024, 11:57:25.302 AM, 11
Incident number 38: 6/26/2024, 3:49:16.784 PM, 4
Incident number 39: 6/27/2024, 2:25:58.353 PM, 48
Incident number 55: 7/2/2024, 9:45:50.756 AM, 22
Incident number 122: 7/13/2024, 9:17:30.965 AM, 9
Incident number 134: 7/17/2024, 10:49:35.108 AM, 6
Incident number 166: 7/22/2024, 8:18:18.418 AM, 9
Incident number 175: 7/23/2024, 12:19:16.000 PM, 31
Incident number 290: 7/29/2024, 10:32:44.433 AM, 46
Incident number 322: 8/1/2024, 12:26:22.000 PM, 9
Incident number 406: 8/6/2024, 5:15:43.000 PM, 22


In [33]:
# unique incident numbers
incident_numbers = filtered_incidents['IncidentNumber'].unique()
# get first entry from each unique incident number
unique_incidents = filtered_incidents.groupby('IncidentNumber').last()

# back to dataframe
unique_incidents = unique_incidents.reset_index()

for i, incident in unique_incidents.iterrows():
    print(f"Incident number {incident['IncidentNumber']}: {incident['FirstActivityTime [UTC]']}, {len(json.loads(incident['AlertIds']))}")

Incident number 3: 6/4/2024, 12:21:22.478 PM, 5
Incident number 5: 6/20/2024, 8:51:07.052 AM, 71
Incident number 34: 6/26/2024, 11:57:25.302 AM, 11
Incident number 38: 6/26/2024, 3:49:16.784 PM, 4
Incident number 39: 6/27/2024, 2:25:58.353 PM, 48
Incident number 55: 7/1/2024, 3:01:28.000 PM, 28
Incident number 122: 7/13/2024, 9:17:30.965 AM, 9
Incident number 134: 7/17/2024, 10:49:35.108 AM, 9
Incident number 166: 7/22/2024, 8:18:18.418 AM, 11
Incident number 175: 7/23/2024, 12:19:16.000 PM, 33
Incident number 290: 7/29/2024, 10:32:44.433 AM, 47
Incident number 322: 8/1/2024, 12:26:22.000 PM, 9
Incident number 406: 8/6/2024, 5:15:43.000 PM, 25


In [16]:
filtered_incidents = pd.DataFrame(filtered_incidents)
filtered_incidents['IncidentNumber'].value_counts()
# get unique incident numbers in a list
incident_numbers = filtered_incidents['IncidentNumber'].unique()

# get incident with number 134
incident_134 = filtered_incidents[filtered_incidents['IncidentNumber'] == 134]

In [20]:
incident_numbers
# get title of incident with different incident numbers
incident_titles = filtered_incidents.groupby('IncidentNumber').last()
for incident_number, title in incident_titles.items():
    print(f"Incident number {incident_number}: {title}")

Incident number 3: BEC financial fraud attack was launched from a compromised account (attack disruption)
Incident number 5: Human-operated ransomware attack was launched from a compromised asset (attack disruption)
Incident number 34: Multi-stage incident involving Execution & Discovery on one endpoint
Incident number 38: Multi-stage incident involving Defense evasion & Discovery on one endpoint
Incident number 39: Multi-stage incident involving Privilege escalation on multiple endpoints reported by multiple sources
Incident number 55: Multi-stage incident involving Initial access & Lateral movement on multiple endpoints reported by multiple sources
Incident number 122: Multi-stage incident involving Initial access & Discovery on one endpoint
Incident number 134: BEC financial fraud attack was launched from a compromised account (attack disruption)
Incident number 166: SAP financial process manipulation (attack disruption)
Incident number 175: Multi-stage incident involving Initial ac

In [21]:
unique_incidents = filtered_incidents.drop_duplicates(subset='IncidentNumber')
len(unique_incidents)

13

In [24]:
unique_incidents.iloc[0]['FirstActivityTime [UTC]']

'6/26/2024, 3:49:16.784 PM'

In [15]:
# iter and print time gen, number of alertids
for i, incident in incident_134.iterrows():
    print(incident['TimeGenerated [UTC]'], len(json.loads(incident['AlertIds'])))

2024-07-17 22:41:41.561000 6
2024-07-18 14:01:51.084000 7
2024-07-18 14:46:52.765000 8
2024-07-18 14:57:07.462000 9
2024-07-19 01:12:43.294000 9
2024-07-19 22:42:11.510000 9
2024-07-20 22:37:12.209000 9
2024-07-21 22:37:24.365000 9
2024-07-22 22:36:59.927000 9
2024-07-23 22:36:35.059000 9
2024-07-24 11:06:31.350000 9
2024-07-24 11:21:48.743000 9
2024-07-24 22:36:50.607000 9
2024-07-25 22:36:47.024000 9
2024-07-26 22:37:08.017000 9
2024-07-27 22:37:15.390000 9
2024-07-28 22:36:24.825000 9
2024-07-29 22:36:50.420000 9
2024-07-30 12:57:02.534000 9
2024-07-30 22:41:39.505000 9
2024-07-31 14:01:28.448000 9
2024-07-31 14:07:08.269000 9
2024-07-31 22:42:18.861000 9
2024-08-01 22:41:59.963000 9
2024-08-02 22:41:43.924000 9
2024-08-03 22:42:28.764000 9
2024-08-04 22:41:48.923000 9
2024-08-05 14:37:13.098000 9
2024-08-05 22:44:20.728000 9
2024-08-06 22:41:32.136000 9
2024-08-07 22:41:30.285000 9
2024-08-08 22:41:25.253000 9
2024-08-09 22:41:38.702000 9
2024-08-10 22:41:50.867000 9
2024-08-11 22: